In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.pipeline import make_pipeline
from category_encoders import BinaryEncoder
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.linear_model import LinearRegression
import xgboost as xgb

In [33]:
data = pd.read_csv('clean_movie_data.csv').drop('Unnamed: 0',axis=1)
data.reset_index(drop=True,inplace=True)
data.head(5)

,title,vote_average,vote_count,release_date,revenue,original_language,runtime,adult,budget,popularity,genres,production_companies,production_countries,spoken_languages
0,Inception,8.364,34495,2010-07-15,825532764,en,148,False,160000000,83.952,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili"
1,Interstellar,8.417,32571,2014-11-05,701729206,en,169,False,165000000,140.241,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English
2,The Dark Knight,8.512,30619,2008-07-16,1004558444,en,152,False,185000000,130.643,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin"
3,Avatar,7.573,29815,2009-12-15,2923706026,en,162,False,237000000,79.932,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish"
4,The Avengers,7.710,29166,2012-04-25,1518815515,en,143,False,220000000,98.082,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian"


In [34]:
data

,title,vote_average,vote_count,release_date,revenue,original_language,runtime,adult,budget,popularity,genres,production_companies,production_countries,spoken_languages
0,Inception,8.364,34495,2010-07-15,825532764,en,148,False,160000000,83.952,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili"
1,Interstellar,8.417,32571,2014-11-05,701729206,en,169,False,165000000,140.241,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English
2,The Dark Knight,8.512,30619,2008-07-16,1004558444,en,152,False,185000000,130.643,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin"
3,Avatar,7.573,29815,2009-12-15,2923706026,en,162,False,237000000,79.932,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish"
4,The Avengers,7.710,29166,2012-04-25,1518815515,en,143,False,220000000,98.082,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8706,Brahmotsavam,4.500,11,2016-05-20,10000000,te,156,False,11000000,1.978,"Family, Romance","G. Mahesh Babu Entertainment Pvt. Ltd, PVP Cinema",India,Telugu
8707,Any Number Can Play,7.000,11,1949-07-15,3205000,en,112,False,1363000,3.364,Drama,Metro-Goldwyn-Mayer,United States of America,English
8708,Pickled Pink,6.727,11,1965-05-12,2436635,en,6,False,2436635,1.399,"Animation, Comedy, Family",DePatie-Freleng Enterprises,United States of America,English
8709,Hollywood & Wine,3.667,11,2011-05-03,8000000,en,95,False,5000000,3.500,"Comedy, Thriller",Aloe Entertainment,United States of America,English


In [35]:
data = data.sample(8711, random_state=0)
data

,title,vote_average,vote_count,release_date,revenue,original_language,runtime,adult,budget,popularity,genres,production_companies,production_countries,spoken_languages
3996,Gran Turismo,8.068,702,2023-08-09,114800000,en,135,False,60000000,2680.593,"Action, Drama, Adventure","PlayStation Productions, 2.0 Entertainment, Co...",United States of America,"English, German, Japanese"
541,Fast & Furious Presents: Hobbs & Shaw,6.867,6618,2019-08-01,760098996,en,137,False,200000000,70.691,"Action, Adventure, Comedy","Chris Morgan Productions, Seven Bucks Producti...",United States of America,"English, Russian, Samoan"
3514,The Outpost,6.885,924,2020-06-24,2133452,en,123,False,18000000,22.097,"War, Drama, History, Action","Millennium Media, York Films, Perfection Hunte...","Bulgaria, United States of America",English
982,Evil Dead,6.500,4226,2013-04-05,97542952,en,91,False,17000000,38.267,Horror,"TriStar Pictures, FilmDistrict, Ghost House Pi...",United States of America,English
394,It Chapter Two,6.854,8021,2019-09-04,473122525,en,169,False,79000000,72.553,"Horror, Fantasy","New Line Cinema, Vertigo Entertainment, Double...",United States of America,English
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4373,Parenthood,6.698,569,1989-07-31,126297830,en,124,False,20000000,13.340,"Comedy, Drama","Universal Pictures, Imagine Entertainment",United States of America,English
7891,Mrs. Soffel,6.107,42,1984-12-26,4385312,en,112,False,11000000,8.634,"Drama, Romance",Metro-Goldwyn-Mayer,United States of America,English
4859,Good Burger,5.999,433,1997-07-25,23700000,en,95,False,8500000,21.959,"Family, Comedy","Paramount, Nickelodeon Movies",United States of America,English
3264,The New World,6.500,1043,2005-12-25,30536013,en,151,False,30000000,15.528,"Drama, History, Romance","The Virginia Company LLC, New Line Cinema, Sun...",United States of America,English


In [36]:
data['release_date'] = pd.to_datetime(data['release_date'])

In [37]:
data['Year'] = data.release_date.dt.year
data['Month'] = data.release_date.dt.month
data['Day'] = data.release_date.dt.day

In [38]:
def get_quarter(df):

    if (df['Month'] <= 3):
        return 'Q1'
    
    elif (df['Month'] <= 6) and (df['Month'] > 3):
        return 'Q2'
              
    elif (df['Month'] <= 9) and (df['Month'] > 6):
        return 'Q3'

    elif (df['Month'] <= 12) and (df['Month'] > 9):
        return 'Q4'

In [39]:
data['Quarter'] = data.apply(get_quarter, axis = 1)

In [40]:
data['YearQuarter'] = data['Year'].astype(str) + data['Quarter'].astype(str)

In [41]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8711 entries, 3996 to 2732
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   title                 8711 non-null   object        
 1   vote_average          8711 non-null   float64       
 2   vote_count            8711 non-null   int64         
 3   release_date          8711 non-null   datetime64[ns]
 4   revenue               8711 non-null   int64         
 5   original_language     8711 non-null   object        
 6   runtime               8711 non-null   int64         
 7   adult                 8711 non-null   bool          
 8   budget                8711 non-null   int64         
 9   popularity            8711 non-null   float64       
 10  genres                8711 non-null   object        
 11  production_companies  8711 non-null   object        
 12  production_countries  8711 non-null   object        
 13  spoken_language

In [42]:
data.head(5)

,title,vote_average,vote_count,release_date,revenue,original_language,runtime,adult,budget,popularity,genres,production_companies,production_countries,spoken_languages,Year,Month,Day,Quarter,YearQuarter
3996,Gran Turismo,8.068,702,2023-08-09,114800000,en,135,False,60000000,2680.593,"Action, Drama, Adventure","PlayStation Productions, 2.0 Entertainment, Co...",United States of America,"English, German, Japanese",2023,8,9,Q3,2023Q3
541,Fast & Furious Presents: Hobbs & Shaw,6.867,6618,2019-08-01,760098996,en,137,False,200000000,70.691,"Action, Adventure, Comedy","Chris Morgan Productions, Seven Bucks Producti...",United States of America,"English, Russian, Samoan",2019,8,1,Q3,2019Q3
3514,The Outpost,6.885,924,2020-06-24,2133452,en,123,False,18000000,22.097,"War, Drama, History, Action","Millennium Media, York Films, Perfection Hunte...","Bulgaria, United States of America",English,2020,6,24,Q2,2020Q2
982,Evil Dead,6.500,4226,2013-04-05,97542952,en,91,False,17000000,38.267,Horror,"TriStar Pictures, FilmDistrict, Ghost House Pi...",United States of America,English,2013,4,5,Q2,2013Q2
394,It Chapter Two,6.854,8021,2019-09-04,473122525,en,169,False,79000000,72.553,"Horror, Fantasy","New Line Cinema, Vertigo Entertainment, Double...",United States of America,English,2019,9,4,Q3,2019Q3


In [43]:
train_data, test_data = train_test_split(data, test_size=0.3, random_state=0)
valid_data, test_data = train_test_split(test_data, test_size=0.5, random_state=0)
train_data.shape[0], valid_data.shape[0], test_data.shape[0]

(6097, 1307, 1307)

In [44]:
label = "revenue"
train_data[label]

3250      4957273
7280       369419
4190     18552314
7108     18591272
2869     91244913
          ...    
921     224834564
2309     51783040
7697       472370
3055     72527595
898     292351275
Name: revenue, Length: 6097, dtype: int64

In [45]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler

import numpy as np

label_pipeline = make_pipeline(
    FunctionTransformer(np.log2, inverse_func=np.exp2),
    StandardScaler(),
)

train_data[label] = label_pipeline.fit_transform(train_data[[label]])
valid_data[label] = label_pipeline.transform(valid_data[[label]])
test_data[label] = label_pipeline.transform(test_data[[label]])

train_data[label]

C:\Users\ETC\AppData\Local\Temp\ipykernel_31888\457155433.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data[label] = label_pipeline.fit_transform(train_data[[label]])


3250   -0.413285
7280   -1.479278
4190    0.128496
7108    0.129357
2869    0.782437
          ...   
921     1.152653
2309    0.549883
7697   -1.378359
3055    0.688190
898     1.260452
Name: revenue, Length: 6097, dtype: float64

In [46]:
features = ['vote_average', 'vote_count', 'original_language', 'runtime', 'adult',
       'budget', 'popularity', 'genres', 'production_companies',
       'production_countries', 'spoken_languages', 'Year', 'Month', 'Day',
       'Quarter', 'YearQuarter']

train_data[features]

,vote_average,vote_count,original_language,runtime,adult,budget,popularity,genres,production_companies,production_countries,spoken_languages,Year,Month,Day,Quarter,YearQuarter
3250,7.158,1051,en,124,False,25000000,12.293,"History, Drama, Mystery, Thriller","Fastnet Films, 22h22, Caviar Antwerp NV, Defin...","Ireland, United States of America, France",English,2019,3,7,Q1,2019Q1
7280,4.796,81,fr,108,False,4800000,9.552,Drama,"Why Not Productions, CNC, Canal+, Pictanovo, C...",France,"Hebrew, French",2022,5,20,Q2,2022Q2
4190,4.102,630,fr,110,False,25868826,10.969,Comedy,Gaumont,France,French,2016,3,23,Q1,2016Q1
7108,5.926,95,en,90,False,4000000,8.903,"Crime, Action, Drama","Universal Pictures, Bigel / Mailer Films",United States of America,"English, Spanish",2002,12,6,Q4,2002Q4
2869,6.639,1284,en,112,False,12000000,20.121,"Romance, Comedy","Pantelion Films, 3Pas Studios, Metro-Goldwyn-M...",United States of America,"English, French, Norwegian, Spanish",2018,4,13,Q2,2018Q2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
921,6.758,4494,en,84,False,45000000,36.884,"Animation, Comedy, Family","DreamWorks Animation, Aardman, Pathé, DreamWor...","United Kingdom, United States of America, France",English,2000,6,23,Q2,2000Q2
2309,6.700,1714,en,136,False,40000000,31.656,"Thriller, Action","Universal Pictures, Bay Films, Endeavor Conten...",United States of America,English,2022,3,16,Q1,2022Q1
7697,6.170,53,en,94,False,150000,3.780,Comedy,Party Productions,United States of America,English,1995,6,9,Q2,1995Q2
3055,5.678,1177,en,100,False,14000000,15.109,"Horror, Thriller","TriStar Pictures, Original Film, Phoenix Pictu...","Canada, France, United States of America",English,1998,9,25,Q3,1998Q3


In [47]:
train_data[features].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6097 entries, 3250 to 898
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   vote_average          6097 non-null   float64
 1   vote_count            6097 non-null   int64  
 2   original_language     6097 non-null   object 
 3   runtime               6097 non-null   int64  
 4   adult                 6097 non-null   bool   
 5   budget                6097 non-null   int64  
 6   popularity            6097 non-null   float64
 7   genres                6097 non-null   object 
 8   production_companies  6097 non-null   object 
 9   production_countries  6097 non-null   object 
 10  spoken_languages      6097 non-null   object 
 11  Year                  6097 non-null   int64  
 12  Month                 6097 non-null   int64  
 13  Day                   6097 non-null   int64  
 14  Quarter               6097 non-null   object 
 15  YearQuarter        

In [48]:
from sklearn.dummy import DummyRegressor

baseline = DummyRegressor(strategy="mean") # DummyClassifier() for classification
baseline.fit(train_data[features], train_data[label])

baseline_score = baseline.score(test_data[features], test_data[label])
print(f"{baseline_score:,.3f}")

-0.000


In [49]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from category_encoders import BinaryEncoder
import xgboost as xgb

def create_pipeline():
    return make_pipeline(
        make_column_transformer(
            (OneHotEncoder(handle_unknown='ignore'), ["genres", 'original_language', 'production_companies',
                               'production_countries', 'spoken_languages', 'YearQuarter']),
            (OneHotEncoder(handle_unknown='ignore'), ['Year', 'Month', 'Day', 'Quarter']),  
            (StandardScaler(), ["vote_average"]),
            (StandardScaler(), ["budget"]),
            (StandardScaler(), ["popularity"]),
            (BinaryEncoder(), ['adult']),
            remainder="passthrough",
        ),
        xgb.XGBRegressor(),  
    )

# Assuming you have train_data, test_data, features, and label
pipeline = create_pipeline()
pipeline.fit(train_data[features], train_data[label])

test_score = pipeline.score(test_data[features], test_data[label])
print(f"{test_score:,.3f}")


0.592


In [50]:
pipeline.fit(train_data[features], train_data[label])

train_score = pipeline.score(train_data[features], train_data[label])
print(f"{train_score:,.3f}")

0.845


In [51]:
pipeline.steps[-1][1].n_features_in_

8627

In [52]:
from sklearn.model_selection import RandomizedSearchCV
params = dict(
    xgbregressor__n_estimators=[1500,2000,2500],
    xgbregressor__learning_rate=[0.01, 0.1],
    xgbregressor__max_depth=[8,9,10,11],
    xgbregressor__gamma=[0, 0.1, 0.2],
    xgbregressor__subsample=[0.6, 0.8, 1.0],
)

search = RandomizedSearchCV(create_pipeline(), params, n_iter=10, cv=5, n_jobs=10, random_state=0)
search.fit(train_data[features], train_data[label])

print(f"{search.best_score_:,.3f}", search.best_params_)

0.646 {'xgbregressor__subsample': 0.6, 'xgbregressor__n_estimators': 2000, 'xgbregressor__max_depth': 8, 'xgbregressor__learning_rate': 0.01, 'xgbregressor__gamma': 0.1}


In [53]:
test_score = search.best_estimator_.score(test_data[features], test_data[label])
print(f"{test_score:,.3f}")

0.629


In [54]:
import joblib

# "Artifacts"
joblib.dump(search.best_estimator_, "pipeline.joblib")
joblib.dump(label_pipeline, "label_pipeline.joblib")
joblib.dump(languages,"languages.joblib")
joblib.dump(available_genres,'available_genres.joblib')
joblib.dump(companies,'companies.joblib')
joblib.dump(production_countrieslist,'production_countrieslist.joblib')
spoken_languages = ['Quechua', 'Tswana', 'Romanian', 'Croatian', 'Georgian', 'French', 'Guarani', 'Punjabi', 'Breton', 'Spanish', 'Navajo', 'Somali', 'Akan', 'Khmer', 'No Language', 'Dutch', 'English', 'Urdu', 'Zulu', 'Mandarin', 'Slovenian', 'Estonian', 'Galician', 'Fulah', 'Welsh', 'Greek', 'Sotho', 'Lao', 'Gujarati', 'Ganda', 'Tamil', 'Icelandic', 'Pushto', 'Yoruba', 'Indonesian', 'Portuguese', 'Burmese', 'Hungarian', 'Bosnian', 'Nepali', 'Arabic', 'German', 'Lingala', 'Maori', 'Kashmiri', 'Swahili', 'Afrikaans', 'Vietnamese', 'Inuktitut', 'Armenian', 'Hindi', 'Hebrew', 'Cornish', 'Kannada', 'Marathi', 'Latin', 'Bambara', 'Tahitian', 'Malay', 'Persian', 'Serbian', 'Slovak', 'Esperanto', 'Tagalog', 'Irish', 'Finnish', 'Lithuanian', 'Hausa', 'Tibetan', 'Ukrainian', 'Amharic', 'Yiddish', 'Albanian', 'Cree', 'Swedish', 'Sanskrit', 'Danish', 'Catalan', 'Serbo-Croatian', 'Chechen', 'Bengali', 'Xhosa', 'Cantonese', 'Italian', 'Corsican', 'Russian', 'Malayalam', 'Telugu', 'Gaelic', 'Tonga', 'Thai', 'Bulgarian', 'Chichewa; Nyanja', 'Polish', 'Maltese', 'Korean', 'Haitian; Haitian Creole', 'Avaric', 'Norwegian', 'Northern Sami', 'Samoan', 'Tatar', 'Turkish', 'Japanese', 'Mongolian', 'Kurdish', 'Sinhalese', 'Czech', 'Basque', 'Ido', 'Moldavian']

# บันทึกข้อมูลลงในไฟล์ "spoken_languages.joblib"
joblib.dump(spoken_languages, "spoken_languages.joblib")
#
#joblib.dump(revenue, "revenue.joblib")

['spoken_languages.joblib']

In [55]:
pipeline = joblib.load("pipeline.joblib")
label_pipeline = joblib.load("label_pipeline.joblib")



pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder-1',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['genres',
                                                   'original_language',
                                                   'production_companies',
                                                   'production_countries',
                                                   'spoken_languages',
                                                   'YearQuarter']),
                                                 ('onehotencoder-2',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Year', 'Month', 'Day',
                                                   'Quarter...
                        

In [60]:
%%writefile app.py
# !pip install gradio ipywidgets
import pandas as pd
import gradio as gr
import re
import joblib

def get_quarter(month):

    if (month <= 3):
        return 'Q1'
    
    elif (month <= 6) and (month > 3):
        return 'Q2'
              
    elif (month <= 9) and (month > 6):
        return 'Q3'

    elif (month <= 12) and (month > 9):
        return 'Q4'
# Load the pipeline and label_pipeline
pipeline = joblib.load("pipeline.joblib")
label_pipeline = joblib.load("label_pipeline.joblib")
available_genres = joblib.load("available_genres.joblib")
companies = joblib.load("companies.joblib")
production_countrieslist = joblib.load("production_countrieslist.joblib")
spoken_languages = joblib.load("spoken_languages.joblib")
languages =joblib.load("languages.joblib")


# Define the predict function
def predict(release_date,vote_average, vote_count, original_language, runtime, adult, budget, popularity, genres, production_companies, production_countries, spoken_languages):
    date_pattern = re.compile(r'^\d{4}-\d{2}-\d{2}$')
    if not date_pattern.match(release_date):
        print("Please enter the release date in the format YYYY-MM-DD.")
    else:
        print(release_date)
        Year, Month, Day = map(int, release_date.split('-'))  # Split the date into Year, Month, and Day
    print(genres)
    sample = dict()
    sample["vote_average"] = float(vote_average)
    sample["vote_count"] = int(vote_count)
    sample["original_language"] =  original_language
    sample["runtime"] = runtime
    sample["adult"] = True if adult == "True" else False
    sample["budget"] = budget
    sample["popularity"] = popularity
    sample["genres"] = ', '.join(genres)
    sample["production_companies"] = ', '.join(production_companies)
    sample["production_countries"] = ', '.join(production_countries)
    sample["spoken_languages"] = ', '.join(spoken_languages)
    sample["Year"] = Year
    sample["Month"] = Month
    sample["Day"] = Day
    sample["Quarter"] = get_quarter(Month)
    sample["YearQuarter"] = str(Year) + get_quarter(Month)

    print(sample["genres"])

    # Make prediction
    revenue = pipeline.predict(pd.DataFrame([sample]))
    revenue = label_pipeline.inverse_transform([revenue])

    return int(revenue[0][0])


# Define the interface
with gr.Blocks() as blocks:
    vote_average = gr.Textbox(label="Vote Average(approximate )")
    vote_count = gr.Number(label="Vote Count(approximate )")
    original_language = gr.Dropdown(label="Original Language", choices=list(languages.keys()))
    runtime = gr.Number(label="Runtime")
    adult = gr.Radio(["True", "False"], label="Adult")
    budget = gr.Number(label="Budget($)")
    popularity = gr.Number(label="Popularity(approximate )")
    genres = gr.Dropdown(available_genres, multiselect=True, label="genres")
    production_companies = gr.Dropdown(companies,multiselect=True,label="Production Companies")
    production_countries = gr.Dropdown(production_countrieslist,multiselect=True,label="Production Countries")
    spoken_languages = gr.Dropdown(spoken_languages,multiselect=True,label="Spoken Languages")
    release_date = gr.Textbox(label="Release Date",placeholder="YYYY-MM-DD") 
    revenue = gr.Number(label="Revenue")

    inputs = [release_date,vote_average, vote_count, original_language, runtime, adult, budget, popularity, genres, production_companies, production_countries, spoken_languages]
    outputs = [revenue]
    
    predict_btn = gr.Button("Predict")
    predict_btn.click(predict, inputs=inputs, outputs=outputs)


if __name__ == "__main__":
    blocks.launch() # Local machine only
    # blocks.launch(server_name="0.0.0.0") # LAN access to local machine
    # blocks.launch(share=True) # Public access to local machine


Overwriting app.py


In [1]:
%run app.py

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
